# Clasificación de Contactos Estrechos de casos positivos de COVID 19.
El Ministerio de Salud de la Provincia de Buenos Aires, promueve la implementación de la identificación precoz y seguimiento de contactos estrechos de personas sospechosas o confirmadas de COVID 19.

Es por ello que desarrolló una plataforma digital integrada, de acceso por internet, a la cual cada usuario accede por habilitación de la solicitud al Ministerio de Salud.

La plataforma asistenciacovid.gba.gob.ar permite:

● Identificar de forma precoz y sencilla de contactos estrechos de casos sospechosos y confirmados.

● Registrar los seguimientos de los contactos estrechos.

● Al ingresar información provincial, permite que el Municipio visualice y tome conocimiento de contactos estrechos en su territorio que son identificados en otro Municipio, y viceversa.

● Interoperar directamente con la plataforma gestioncovid.gba.gob.ar, de manera que si un contacto pasa a ser sospechoso, éste es visualizado en tiempo real por el nivel Municipal, sin importar desde dónde se haga el registro (CAPS, domicilio de integrante del equipo de salud o CeTeC Municipal o Universitario).
La definición y desarrollo de acciones de aislamiento y seguimiento específicas para el caso de contactos estrechos se fundamenta en limitar tempranamente el contacto físico de personas infectadas, para disminuir la transmisión persona a persona, siendo una de las medidas más eficaces para prevenir la propagación de la enfermedad. Este es el trabajo de, entre otros, Centros de Telemedicina COVID 19, uno de los cuales se encuentra operando en la Facultad de Ciencias Exactas y Naturales de la Universidad de Buenos Aires. Dicha sede se encarga específicamente de trabajar junto a los Municipios de Tigre y Vicente Lopez, ubicados en la Zona Norte del Gran Buenos Aires.

Se nos ocurre que, además de la pronta identificación y aislamiento de contactos estrechos, resulta pertinente- sobretodo en plena segunda ola, cuando las UTI están saturadas y conseguir una cama para internación resulta imposible- la clasificación de éstos en base a su exposición al virus y demás factores que podrían devenir en un mayor riesgo, o no, de haberse contagiado, con el fin de poder destinar los recursos de mejor manera. 

***Objetivo***: En el siguiente trabajo fabricaremos un clasificador para contactos estrechos (CE) de casos positivos de Covid 19. 
Se utilizarán datos obtenidos de la base de datos de la página asistenciacovid.gob.ar, específicamente los de los Municipios de Vicente Lopez y Tigre con los que trabaja la Facultad desde el 1 de Enero de 2021 hasta el 31 de Mayo del idem.


**Paso 1:** Cargo todos los módulos que utilizaré a lo largo del proyecto.

In [ ]:
#Importo los módulos de pandas y numpy.
import pandas as pd
import numpy as np
#Importo el módulo para leer archivos Excel.
import xlrd
#Importo el módulo OS.
import os

**Paso dos:** genero los dataframes que contendrán la información para ambos Municipios.

***Para el Municipio de Tigre***

In [ ]:
#Monto el drive.
from google.colab import drive
drive.mount('/content/drive') 

#Copio la location de todos los files de Tigre que vamos a estar queriendo abrir y convertir eventualmente en un mega dataframe.
location = '/content/drive/My Drive/Covidata_team/Planillas_tigre_excel' 

#Seteo la location como directorio para ir a buscar ahí a todos los archivos.
cwd = os.chdir(location)
#Enlisto todos los nombres de los archivos que pertenecen al directorio declarado.
files = os.listdir(cwd) 

#Todos los archivos son planillas de cálculo de Excel que contienen numerosas hojas. 
#Tres de ellas son las que contienen información de pacientes sospechosos de COVID, es decir quienes se hayan realizado hisopados,
#la de contactos estrechos, que son quienes han sido declarados como contactos según el protocolo de GBA y a quienes se les hace
#un seguimiento telefónico para evaluar sintomatología y necesidades urgentes durante su aislamiento domiciliario, y finalmente
#la de inactivos, que son personas que o bien ya terminaron de transitar la enfermedad o se las dio de baja en la plataforma por
#no poder ser comunicados durante la totalidad del ciclo de vida del virus. 

#Genero 2 dataframes vacíos.
#Uno contendrá los casos sospechosos y el otro contendrá los contactos estrechos. La idea será volcar la información de las planillas en el dataframe correspondiente. 
sospechosos_tigre = pd.DataFrame()
CEs_tigre = pd.DataFrame()
inactivos_tigre = pd.DataFrame()

#La idea ahora es recorrer toda la carpeta e ir pegando archivo por archivo en los dataframes correspondientes la información contenida en ellos.
for i in files: #Por cada archivo en la carpeta de interés.
   if i.endswith('.xlsx'):  #Si es un archivo de planilla de cálculo.
     sospechosos_iterado_tigre = pd.read_excel(i, sheet_name=0, header=0, index_col=None, keep_default_na=True)
     sospechosos_tigre = sospechosos_tigre.append(sospechosos_iterado_tigre, ignore_index = True)
     CEs_iterado_tigre = pd.read_excel(i, sheet_name=1, header=0, index_col=None, keep_default_na=True)
     CEs_tigre = CEs_tigre.append(CEs_iterado_tigre, ignore_index = True)
     inactivos_iterado_tigre = pd.read_excel(i, sheet_name=2, header=0, index_col=None, keep_default_na=True)
     inactivos_tigre = inactivos_tigre.append(inactivos_iterado_tigre, ignore_index = True)

Mounted at /content/drive


In [ ]:
CEs_tigre


,Nombre,Apellido,documento,Info contacto,Teléfono,Domicilio,Email,Nacimiento,Dias_con_sintomas,Ultimo Contacto,Fecha_apertura,Establecimiento,Seguimientos_pendientes,Clasificacion,Observación
0,MARIO ALBERTO,PEREYRA,33835826,1131393161,NaN,BOULOGNE SOUMER 1393,NaN,1987-12-17,331,NaN,2020-06-27,NaN,SIN SEGUIMIENTO,Contacto Estrecho,NaN
1,GISELLE LUANA,LEDESMA,42236718,,NaN,LAS DALIAS 2305,capslapaloma@gmail.com,2000-01-08,328,NaN,2020-06-08,Centro de Salud Rincón,SIN SEGUIMIENTO,Contacto Estrecho,\n --- fecha de inicio de sintomas mal cargada...
2,MELISA NEREA,GONZALEZ,35754800,,1164529892,URUGUAY 1879,capslapaloma@tigre.gob.ar,1991-07-25,327,2021-02-21T00:00:00-0300,2020-06-24,Unidad Sanitaria La Paloma,75,Contacto Estrecho,necesita el certificado de aislamiento
3,ELSA LUISA,TUMBARELLO,18553230,,NaN,ECUADOR 1510,NaN,1967-08-17,294,NaN,2020-07-23,NaN,SIN SEGUIMIENTO,Contacto Estrecho,\n --- fecha de inicio de sintomas mal cargada...
4,VICTOR EZEQUIEL,GONZALEZ,32255226,1551140597,1130677209,CERVIÑO 2836,NaN,1986-01-26,287,NaN,2020-07-30,NaN,SIN SEGUIMIENTO,Contacto Estrecho,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280260,florencia,villaverde,468112475,NaN,1540508296,GUIRALDES 2752,NaN,2021-05-28,--,2021-05-26T00:00:00-0300,NaT,Hospital Materno Infantil Dr. Florencio Escardo,1,Contacto Estrecho,"Nietx de claudio(+). Viven con Marta (+, mamá ..."
280261,Chiara,Villaverde,468112474,NaN,1540508296,GUIRALDES 2752,NaN,2021-05-28,--,2021-05-26T00:00:00-0300,NaT,Hospital Materno Infantil Dr. Florencio Escardo,1,Contacto Estrecho,"Nietx de claudio(+). Viven con Marta (+, mamá ..."
280262,marta,cancino,468112471,NaN,1540508296,GUIRALDES 2752,NaN,2021-05-28,--,2021-05-26T00:00:00-0300,NaT,Hospital Materno Infantil Dr. Florencio Escardo,1,Contacto Estrecho,"Nietx de claudio(+). Viven con Marta (+, mamá ..."
280263,Candela,Villaverde,468112473,NaN,1540508296,GUIRALDES 2752,NaN,2021-05-28,--,2021-05-27T00:00:00-0300,NaT,Hospital Materno Infantil Dr. Florencio Escardo,1,Contacto Estrecho,"Nietx de claudio(+). Viven con Marta (+, mamá ..."


Voy a quedarme sólo con algunas columnas de los DFs originales, eliminando toda la información que pareciera no sumar al análisis que vamos a querer hacer. Así que recortaré el DF para quedarme sólo con los nombres, números de documento, domicilios, fechas de nacimiento, fecha de inicio de síntomas, clasificación del paciente (Sospechoso, Confirmado, Descartado), fecha de último contacto, la cantidad de seguimientos pendientes y las observaciones (una columna en la que los operadores de los centros pueden escribir observaciones del caso, feature fundamental sobretodo para los contactos estrechos).

In [ ]:
sospechosos_tigre = sospechosos_tigre[['Nombre', 'Apellido', 'documento', 'Domicilio', 'Nacimiento', 'FApertura', 'FISintomas', 'Clasificacion', 'Observación']].copy()
CEs_tigre = CEs_tigre[['Nombre', 'Apellido', 'documento', 'Teléfono', 'Domicilio', 'Nacimiento', 'Dias_con_sintomas', 'Ultimo Contacto', 'Establecimiento', 'Seguimientos_pendientes', 'Clasificacion', 'Observación']].copy()
inactivos_tigre = inactivos_tigre[['Nombre', 'Apellido', 'documento', 'Teléfono', 'Domicilio', 'Nacimiento', 'Dias_con_sintomas', 'Ultimo Contacto', 'Establecimiento', 'Seguimientos_pendientes', 'Clasificacion', 'Observación']].copy()

El problema al que me enfrento ahora es que la información en las planillas puede estar repetida.

Por ejemplo, en el caso de los sospechosos, éstos pueden estar repetidos si no logran ser contactados telefónicamente para hacer la búsqueda de sus contactos estrechos. El protocolo del CeTeC llama múltiples veces a lo largo de varios días hasta que el sospechoso contesta. En caso contrario, se suspende la búsqueda para dicha persona. También, en el caso de los contactos estrechos, éstos siempre figurarán repetidamente en la planilla a lo largo de un par de días pues la función del CeTec es hacer un seguimiento de forma diaria o día por medio de sus síntomas o necesidades durante el aislamiento en sus domicilios. 

Por lo tanto, acondicionaré los dataframes de manera tal que figure sólo 1 fila de la información del contacto estrecho en cuestión. 

En particular, como a la hora de hacer los llamados de averiguación de contactos estrechos o seguimientos a dichos CEs muchas veces se encuentra información que pudiera ser relevante y como los trabajadores del CeTeC depositan esta información actualizándola día a día en la columna de Observaciones, me quedo sólo con la última fila para dichos casos, siguiendo esos tiempos. 

**Paso 3:** acondicionamiento de los data frames.

In [ ]:
#Lo que voy a querer hacer ahora es eliminar las filas duplicadas. 
#Voy a hacer que el método conserve sólo la última aparición del número de DNI. 
sospechosos_tigre = sospechosos_tigre.drop_duplicates(subset='documento', keep='last', inplace=False, ignore_index=False)
CEs_tigre = CEs_tigre.drop_duplicates(subset='documento', keep='last', inplace=False, ignore_index=False)
inactivos_tigre = inactivos_tigre.drop_duplicates(subset='documento', keep='last', inplace=False, ignore_index=False)

In [ ]:
len(sospechosos_tigre[sospechosos_tigre["Clasificacion"]=="Sospechoso"])

24065

In [ ]:
sospechosos_tigre

,Nombre,Apellido,documento,Domicilio,Nacimiento,FApertura,FISintomas,Clasificacion,Observación
2,ANABELA,RODRIGUEZ,21637695,ECUADOR 845,1971-01-20,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
17,OSVALDO JORGE,GARIBALDI,8519509,COLOMBRES 80,1951-03-30,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
26,JULIANA,CALLEJON,37685874,ARROYO ANGOSTURA 0,1991-07-21,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
30,CRISTIAN GUILLERMO RAUL,GUZMAN,29171235,AUSTRALIA 2983,1981-11-05,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
42,MARIA EMILIA,MARCHIORI,44725230,CHILE 3265,2003-03-10,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
...,...,...,...,...,...,...,...,...,...
436362,TEODORA EDUARDA,ARELLANO TRINIDAD,19039486,LOS ALELIES 2175,1956-10-12,2021-05-26,NaT,Confirmado,\n --- fecha de inicio de sintomas mal cargada
436363,CARLOS ADALBERTO,BALDUNCIEL,16476171,GODOY CRUZ 1149,1963-08-06,2021-05-28,NaT,Confirmado,\n --- fecha de inicio de sintomas mal cargada
436364,NADIA GISELA,CASTELO,29436449,MARCOS SASTRE 2492,1982-03-24,2021-05-27,NaT,Confirmado,\n --- fecha de inicio de sintomas mal cargada
436365,PATRICIA LAURA,SANTIAGO,22847991,DEL CASSE 3204,1972-07-25,2021-05-28,NaT,Confirmado,\n --- fecha de inicio de sintomas mal cargada


In [ ]:
len(inactivos_tigre[~inactivos_tigre.documento.isin(CEs_tigre.documento)])


439

In [ ]:
inactivos_tigre[~inactivos_tigre.documento.isin(CEs_tigre.documento)]

,Nombre,Apellido,documento,Teléfono,Domicilio,Nacimiento,Dias_con_sintomas,Ultimo Contacto,Establecimiento,Seguimientos_pendientes,Clasificacion,Observación
4633,Abril,ORLANDINI,4.26606e+07,1167106534,Bernárdez Ylorca,2021-05-06,--,NaN,Hospital Municipal San José,2,Contacto Estrecho,FUC: 2/5.
8114,Juana,Ciseros,4.46642e+06,1137745122,CHILE 978,1942-06-24,--,NaN,Hospital Municipal San José,10,Contacto Estrecho,Pasa a sospechoso porque presento fiebre en el...
8115,Camila,Benites,2.3236e+11,2323604511,no se,2021-04-15,--,NaN,Hospital Municipal San José,-,Contacto Estrecho,FALTA primer seguimiento. MAL FUC DNI y fecha ...
8116,Emanuel,Ehessel,3.36518e+07,1133315833,GUIDO SPANO 53,1987-09-21,--,NaN,Hospital Municipal San José,-,Contacto Estrecho,NaN
8117,Joac Yair,Ehessel,1.12738e+11,1127375568,GUIDO SPANO 53,2021-04-15,--,NaN,Hospital Municipal San José,-,Contacto Estrecho,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
33348,Hugo Ricardo,ALBORNOZ,8.48741e+06,1562476037,RICARDO BALBIN 1240,1950-10-20,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,-,Contacto Estrecho,FUC: 17/05
33349,Joaquin Nicolas,DA SILVA,4.62793e+07,1562476037,RICARDO BALBIN 1240,2004-02-12,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,-,Contacto Estrecho,FUC: 17/05
33350,CANDELARIA,RONDAN,5.48324e+09,1562476037,RICARDO BALBIN 1241,2015-06-01,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,-,Contacto Estrecho,FUC: 17/06 Verificar DNI erroneo
33352,Natalia Lorena,ALBORNOZ,2.61242e+07,1562476037,RICARDO BALBIN 1242,1977-08-09,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,-,Contacto Estrecho,FUC: 17/05


In [ ]:
inactivos_tigre['Clasificacion'].unique()

array(['Confirmado', 'Contacto Estrecho', 'Descartado', 'Sospechoso'],
      dtype=object)

In [ ]:
CEs_tigre.to_csv('CEs_tigre.csv')
!cp CEs_tigre.csv "/content/drive/My Drive/Covidata_team/"

**Paso 4:** Obtención de las labels


In [ ]:
CEs_filtrados = CEs_tigre[CEs_tigre["documento"].isin(list(sospechosos_tigre["documento"])) | CEs_tigre["documento"].isin(list(inactivos_tigre["documento"]))].copy()

In [ ]:
df_sospechosos = sospechosos_tigre[["documento","Clasificacion"]].copy()
df_inactivos =  inactivos_tigre[["documento","Clasificacion"]].copy()
df_filtrados = CEs_filtrados[["documento","Nacimiento","Ultimo Contacto","Observación"]].copy()


**df_sospechosos**

In [ ]:
display(df_sospechosos.head())

,documento,Clasificacion
2,21637695,Sospechoso
17,8519509,Sospechoso
26,37685874,Sospechoso
30,29171235,Sospechoso
42,44725230,Sospechoso


**df_inactivos**

In [ ]:
display(df_inactivos.head())

,documento,Clasificacion
1193,9.23843e+07,Confirmado
1345,2.36501e+07,Contacto Estrecho
2003,5.78982e+07,Contacto Estrecho
3488,3.09922e+07,Confirmado
4482,1.81928e+07,Contacto Estrecho


In [ ]:
df_filtrados_sospechosos = pd.merge(df_filtrados,df_sospechosos,on="documento")

In [ ]:
display(df_filtrados_sospechosos.head())
len(df_filtrados_sospechosos)

,documento,Nacimiento,Ultimo Contacto,Observación,Clasificacion
0,39593772,1996-05-08,NaN,\n --- fecha de inicio de sintomas mal cargada,Confirmado
1,40624162,1997-03-10,NaN,ALISON SE HISOPO Y DIO DESCARTADA (APARECE EN ...,Sospechoso
2,23314897,NaT,NaN,SINTOMAS (día de inicio:): no presenta\nSE AUT...,Confirmado
3,16699966,2021-04-16,2021-04-16T00:00:00-0300,16/4: es la madre de la sospechosa. Está basta...,Sospechoso
4,21683972,1971-02-05,2021-04-29T00:00:00-0300,SIN SÍNTOMAS. CE PERMANENTE. ES LA HIJA,Sospechoso


227

In [ ]:
df_filtrados_inactivos = pd.merge(df_filtrados,df_inactivos,on="documento")

In [ ]:
display(df_filtrados_inactivos)
len(df_filtrados_inactivos)

,documento,Nacimiento,Ultimo Contacto,Observación,Clasificacion
0,21937881,1971-01-10,2021-04-30T00:00:00-0300,"\nHija de maria alicia (+), no se aislan entre...",Descartado
1,38992818,1977-05-12,2021-05-09T00:00:00-0300,Convive con el hermano (+). Se aislan. FUC:9/5...,Contacto Estrecho
2,11228779282,2019-10-02,2021-05-18T00:00:00-0300,NaN,Contacto Estrecho
3,95250767,1983-04-18,2021-05-18T00:00:00-0300,NaN,Contacto Estrecho
4,11228779281,2021-05-18,2021-05-18T00:00:00-0300,NaN,Contacto Estrecho
...,...,...,...,...,...
2062,11111111,2020-11-03,2020-11-06T00:00:00-0300,04/11/2020 Contacto fallido (turno mañana)\n04...,Contacto Estrecho
2063,123456123456,NaT,2020-10-22T00:00:00-0300,NaN,Contacto Estrecho
2064,111111,2020-11-20,2020-09-29T00:00:00-0300,NaN,Contacto Estrecho
2065,35913160,NaT,NaN,SÍNTOMAS: No.\nFIEBRE: No.\n¿SE AUTOMEDICO?: N...,Contacto Estrecho


2067

In [ ]:
df_integrado = pd.concat([df_filtrados_sospechosos,df_filtrados_inactivos])

In [ ]:
#df_integrado2 = df_integrado.drop_duplicates(subset="documento")
#df_integrado2 = df_integrado2.dropna(subset=["Observación"])
print(len(df_integrado2[df_integrado2["Clasificacion"]=="Sospechoso"]))
df_integrado2["Clasificacion"].unique()

138


array(['Confirmado', 'Sospechoso', nan, 'Descartado', 'Contacto Estrecho'],
      dtype=object)

In [ ]:
df_observaciones = df_integrado[(df_integrado["Clasificacion"]=="Confirmado") | (df_integrado["Clasificacion"]=="Descartado")]

In [ ]:
df_observaciones = df_observaciones.dropna(subset=["Observación"]).drop_duplicates(subset=["documento"])


In [ ]:
Limpieza de los dataframes


Obtenemos el df de contactos estrechos con las columnas de interés

In [ ]:
df_CEs = CEs_tigre[["documento","Nacimiento","Ultimo Contacto","Clasificacion","Observación"]].copy()

Chequeamos qué tenemos en la clasificación

In [ ]:
df_CEs["Clasificacion"].unique()

array(['Contacto Estrecho'], dtype=object)

Como son todos CEs, no nos interesa esta columna.

In [ ]:
df_CEs = df_CEs[["documento","Nacimiento","Ultimo Contacto","Observación"]]

Repetimos la limpieza con el de sospechosos


In [ ]:
sospechosos_tigre.head()

,Nombre,Apellido,documento,Domicilio,Nacimiento,FApertura,FISintomas,Clasificacion,Observación
2,ANABELA,RODRIGUEZ,21637695,ECUADOR 845,1971-01-20,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
17,OSVALDO JORGE,GARIBALDI,8519509,COLOMBRES 80,1951-03-30,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
26,JULIANA,CALLEJON,37685874,ARROYO ANGOSTURA 0,1991-07-21,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
30,CRISTIAN GUILLERMO RAUL,GUZMAN,29171235,AUSTRALIA 2983,1981-11-05,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada
42,MARIA EMILIA,MARCHIORI,44725230,CHILE 3265,2003-03-10,2021-05-09,NaT,Sospechoso,\n --- fecha de inicio de sintomas mal cargada


In [ ]:
df_sospechosos = sospechosos_tigre[["documento","FApertura","FISintomas","Clasificacion","Observación"]].copy()

Verificamos que no nos interesan las observaciones pues son inválidas.

In [ ]:
df_sospechosos["Observación"].unique()

array(['\n --- fecha de inicio de sintomas mal cargada ',
       'Solicitado por guardia Sanatorio San Lucas\n --- fecha de inicio de sintomas mal cargada ',
       'TEL 1149702517\n --- fecha de inicio de sintomas mal cargada 03/05/2021',
       ..., '22965423N\n --- fecha de inicio de sintomas mal cargada ',
       'fiebre dolor de garganta y dificultad respiratpria\n --- fecha de inicio de sintomas mal cargada ',
       'test de antigeno 19/5 positivo\n --- fecha de inicio de sintomas mal cargada '],
      dtype=object)

In [ ]:
df_sospechosos = sospechosos_tigre[["documento","FApertura","Clasificacion"]].copy()

Veamos qué pinta tienen las clasificaciones

In [ ]:
df_sospechosos["Clasificacion"].value_counts()

Sospechoso    24065
Confirmado    21551
Descartado        3
Name: Clasificacion, dtype: int64

Al ver estos resultados y con el contexto que tenemos, sabemos que los sospechosos pueden contar como descartados.

In [ ]:
df_sospechosos.loc[df_sospechosos['Clasificacion'] == "Sospechoso", "Clasificacion"] = "Descartado"
df_sospechosos["Clasificacion"].value_counts()

Descartado    24068
Confirmado    21551
Name: Clasificacion, dtype: int64

In [ ]:
inactivos_tigre.head()

,Nombre,Apellido,documento,Teléfono,Domicilio,Nacimiento,Dias_con_sintomas,Ultimo Contacto,Establecimiento,Seguimientos_pendientes,Clasificacion,Observación
1193,RICHARD WASHINGTON,AYRALA,9.23843e+07,1168017085,WILLIAMS 22,1961-12-06,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,11,Confirmado,\n --- fecha de inicio de sintomas mal cargada
1345,Walter,Tolosa,2.36501e+07,1554687825,pedir,2021-04-22,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,-,Contacto Estrecho,"Hacer 1º seguimiento, FUC: 16/4? chequear (Vín..."
2003,Emilia,MARTINEZ,5.78982e+07,1124863722,FRAY M ESQUIU 716,2019-09-13,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,6,Contacto Estrecho,El padre es (+) y la madre (Lucia) también. \n...
3488,RAUL RICARDO,PEREYRA,3.09922e+07,1140419753,VICTOR HUGO 2217,1984-08-02,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,12,Confirmado,\n --- fecha de inicio de sintomas mal cargada
4482,Monica Isabel,PEREYRA,1.81928e+07,47273123,CAMACUA 3789,1967-03-06,--,NaN,Hospital Materno Infantil Dr. Florencio Escardo,2,Contacto Estrecho,FUC 25/4. Esposa conviviente. Preguntar por el...


In [ ]:
df_inactivos = inactivos_tigre[["documento","Clasificacion","Observación"]].copy()

In [ ]:
sospechosos_y_CE = pd.merge(df_sospechosos,df_CEs,on="documento")

Sacamos los NA molestos para hacer análisis de texto

In [ ]:
sospechosos_y_CE = sospechosos_y_CE.dropna(subset=["Observación","Clasificacion"])

In [ ]:
sospechosos_y_CE.head()

,documento,FApertura,Clasificacion,Nacimiento,Ultimo Contacto,Observación
0,43087241,2021-05-08,Descartado,2001-02-20,2020-12-17T00:00:00-0300,"Atiende gabriela salom (positivo, le dan el al..."
1,40762085,2021-04-16,Descartado,1994-11-10,2021-04-27T00:00:00-0300,\nno se aisla del caso conf.
2,36553715,2021-04-22,Confirmado,1992-01-21,NaN,acompañante
3,42419804,2021-03-31,Descartado,2000-02-23,2021-03-26T00:00:00-0300,Alta: 9/4 (Se aisla del vínculo). El trabajo l...
4,94451766,2021-03-23,Descartado,1983-05-20,2021-04-15T00:00:00-0300,\nFUC: 15/4\nManzo miguel ángel el 10/04/2021 ...


Ahora hacemos el merge de CE e inactivos

In [ ]:
inactivos_y_CE = pd.merge(df_inactivos,df_CEs,on="documento")

In [ ]:
inactivos_y_CE

,documento,Clasificacion,Observación_x,Nacimiento,Ultimo Contacto,Observación_y
0,9.23843e+07,Confirmado,\n --- fecha de inicio de sintomas mal cargada,1961-12-06,2021-04-16T00:00:00-0300,FUC: 15/4. Esperando para hisoparse (tuvo dolo...
1,2.36501e+07,Contacto Estrecho,"Hacer 1º seguimiento, FUC: 16/4? chequear (Vín...",2021-04-22,2021-04-16T00:00:00-0300,"Hacer 1º seguimiento, FUC: 16/4? chequear (Vín..."
2,5.78982e+07,Contacto Estrecho,El padre es (+) y la madre (Lucia) también. \n...,2019-09-13,2021-05-04T00:00:00-0300,El padre es (+) y la madre (Lucia) también. \n...
3,3.09922e+07,Confirmado,\n --- fecha de inicio de sintomas mal cargada,1984-08-02,2021-04-17T00:00:00-0300,"FUC: pedir alta Alexis (sospechoso, no se aísl..."
4,1.81928e+07,Contacto Estrecho,FUC 25/4. Esposa conviviente. Preguntar por el...,1967-03-06,2021-04-25T00:00:00-0300,FUC 25/4. Esposa conviviente. Preguntar por el...
...,...,...,...,...,...,...
2062,5.04888e+07,Confirmado,\n --- fecha de inicio de sintomas mal cargada,2021-05-17,2021-05-17T00:00:00-0300,Atiende Braian (+) padre de quiara. No se aisl...
2063,5.82115e+08,Contacto Estrecho,Atiende Braian (+) padre de Enzo (tiene 6 mese...,2021-05-17,2021-05-17T00:00:00-0300,Atiende Braian (+) padre de Enzo (tiene 6 mese...
2064,4.97399e+07,Contacto Estrecho,Atiende Christian Greppi. FUC: 16/5 (alta de a...,2009-09-15,2021-05-16T00:00:00-0300,Atiende Christian Greppi. FUC: 16/5 (alta de a...
2065,4.56827e+07,Contacto Estrecho,Atiende Noemi. FUC: 18/5 (alta de la madre +)....,2004-03-15,2021-05-17T00:00:00-0300,Atiende Noemi. FUC: 18/5 (alta de la madre +)....


In [ ]:
len(inactivos_y_CE[inactivos_y_CE["Clasificacion"]!="Contacto Estrecho"])

212

Verificamos que las observaciones no existentes en y tampoco existen en x. Nos podemos quedar solo con observacion X


In [ ]:
inactivos_y_CE[inactivos_y_CE["Observación_y"] == "\n --- fecha de inicio de sintomas mal cargada "]

,documento,Clasificacion,Observación_x,Nacimiento,Ultimo Contacto,Observación_y
348,36689322,Descartado,\n --- fecha de inicio de sintomas mal cargada,1992-02-08,2021-01-15T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
364,20628467,Contacto Estrecho,\n --- fecha de inicio de sintomas mal cargada,1969-06-17,2021-01-18T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
373,29382697,Confirmado,\n --- fecha de inicio de sintomas mal cargada,1982-04-26,2021-01-27T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
409,3.61485e+07,Confirmado,\n --- fecha de inicio de sintomas mal cargada...,1991-04-02,2021-01-25T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
421,2.17865e+07,Contacto Estrecho,\n --- fecha de inicio de sintomas mal cargada,1970-11-20,2021-02-02T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
450,2.56788e+07,Descartado,\n --- fecha de inicio de sintomas mal cargada,1977-01-20,2021-01-30T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
571,4.44822e+07,Descartado,\n --- fecha de inicio de sintomas mal cargada,2002-09-20,2021-02-19T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
586,2.74289e+07,Contacto Estrecho,\n --- fecha de inicio de sintomas mal cargada,1979-08-19,2021-02-23T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
629,1.82055e+07,Descartado,\n --- fecha de inicio de sintomas mal cargada,1966-10-16,2021-02-27T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada
831,2.00288e+07,Descartado,\n --- fecha de inicio de sintomas mal cargada,1968-04-01,2021-03-22T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada


In [ ]:
inactivos_y_CE = inactivos_y_CE[["documento","Clasificacion","Observación_y","Ultimo Contacto","Nacimiento"]]

In [ ]:
inactivos_y_CE = inactivos_y_CE.rename(columns={"Observación_y":"Observación"})

In [ ]:
inactivos_y_CE.head()

,documento,Clasificacion,Observación,Ultimo Contacto,Nacimiento
0,9.23843e+07,Confirmado,FUC: 15/4. Esperando para hisoparse (tuvo dolo...,2021-04-16T00:00:00-0300,1961-12-06
1,2.36501e+07,Contacto Estrecho,"Hacer 1º seguimiento, FUC: 16/4? chequear (Vín...",2021-04-16T00:00:00-0300,2021-04-22
2,5.78982e+07,Contacto Estrecho,El padre es (+) y la madre (Lucia) también. \n...,2021-05-04T00:00:00-0300,2019-09-13
3,3.09922e+07,Confirmado,"FUC: pedir alta Alexis (sospechoso, no se aísl...",2021-04-17T00:00:00-0300,1984-08-02
4,1.81928e+07,Contacto Estrecho,FUC 25/4. Esposa conviviente. Preguntar por el...,2021-04-25T00:00:00-0300,1967-03-06


In [ ]:
inactivos_y_CE = inactivos_y_CE.dropna(subset=["Observación","Clasificacion"])

In [ ]:
inactivos_y_CE["Clasificacion"].value_counts()

Contacto Estrecho    1568
Descartado             94
Confirmado             66
Sospechoso              5
Name: Clasificacion, dtype: int64

In [ ]:
inactivos_y_CE.loc[inactivos_y_CE['Clasificacion'] == "Sospechoso", "Clasificacion"] = "Descartado"

In [ ]:
inactivos_y_CE["Clasificacion"].value_counts()

Contacto Estrecho    1568
Descartado             99
Confirmado             66
Name: Clasificacion, dtype: int64

In [ ]:
inactivos_y_CE = inactivos_y_CE[inactivos_y_CE["Clasificacion"] != "Contacto Estrecho"]

In [ ]:
len(inactivos_y_CE)

165

In [ ]:
inactivos_y_CE["Clasificacion"].value_counts()

Descartado    99
Confirmado    66
Name: Clasificacion, dtype: int64

In [ ]:
df_final = pd.concat([sospechosos_y_CE,inactivos_y_CE])

294

In [ ]:
df_final = df_final.drop_duplicates(subset=["documento"],keep="first").copy()

In [ ]:
len(df_final)

294

In [ ]:
df_final

,documento,FApertura,Clasificacion,Nacimiento,Ultimo Contacto,Observación
0,43087241,2021-05-08,Descartado,2001-02-20,2020-12-17T00:00:00-0300,"Atiende gabriela salom (positivo, le dan el al..."
1,40762085,2021-04-16,Descartado,1994-11-10,2021-04-27T00:00:00-0300,\nno se aisla del caso conf.
2,36553715,2021-04-22,Confirmado,1992-01-21,NaN,acompañante
3,42419804,2021-03-31,Descartado,2000-02-23,2021-03-26T00:00:00-0300,Alta: 9/4 (Se aisla del vínculo). El trabajo l...
4,94451766,2021-03-23,Descartado,1983-05-20,2021-04-15T00:00:00-0300,\nFUC: 15/4\nManzo miguel ángel el 10/04/2021 ...
...,...,...,...,...,...,...
1997,9.25437e+07,NaT,Descartado,2021-05-21,2021-05-19T00:00:00-0300,FUC 19/5. La hija y su ex mujer dieron +. Ella...
2006,4.29082e+07,NaT,Confirmado,2021-05-21,2021-05-20T00:00:00-0300,"Atiende Angela, esposa de marcelo (+), madre d..."
2039,3.8007e+07,NaT,Descartado,1994-04-26,2021-05-22T00:00:00-0300,FUC: 22/05. Son compañeres del trabajo de dani...
2050,3.31961e+07,NaT,Descartado,1987-07-28,2021-05-27T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada


In [ ]:
df_final.loc[df_final['Clasificacion'] == "Descartado", "Clasificacion"] = 0
df_final.loc[df_final['Clasificacion'] == "Confirmado", "Clasificacion"] = 1

In [ ]:
df_final

,documento,FApertura,Clasificacion,Nacimiento,Ultimo Contacto,Observación
0,43087241,2021-05-08,0,2001-02-20,2020-12-17T00:00:00-0300,"Atiende gabriela salom (positivo, le dan el al..."
1,40762085,2021-04-16,0,1994-11-10,2021-04-27T00:00:00-0300,\nno se aisla del caso conf.
2,36553715,2021-04-22,1,1992-01-21,NaN,acompañante
3,42419804,2021-03-31,0,2000-02-23,2021-03-26T00:00:00-0300,Alta: 9/4 (Se aisla del vínculo). El trabajo l...
4,94451766,2021-03-23,0,1983-05-20,2021-04-15T00:00:00-0300,\nFUC: 15/4\nManzo miguel ángel el 10/04/2021 ...
...,...,...,...,...,...,...
1997,9.25437e+07,NaT,0,2021-05-21,2021-05-19T00:00:00-0300,FUC 19/5. La hija y su ex mujer dieron +. Ella...
2006,4.29082e+07,NaT,1,2021-05-21,2021-05-20T00:00:00-0300,"Atiende Angela, esposa de marcelo (+), madre d..."
2039,3.8007e+07,NaT,0,1994-04-26,2021-05-22T00:00:00-0300,FUC: 22/05. Son compañeres del trabajo de dani...
2050,3.31961e+07,NaT,0,1987-07-28,2021-05-27T00:00:00-0300,\n --- fecha de inicio de sintomas mal cargada


In [ ]:
location = '/content/drive/My Drive/Covidata_team/df_final.csv' 
df_final.to_csv(location)